# Hyperparameters_tunning

1.GridSearch

2.Random Search

3.Bayesian Optimization

4.Optuna

### here are several methods for hyperparameter tuning:

Grid Search: Grid search involves specifying a set of values for each hyperparameter and then exhaustively searching all possible combinations of these values. This method is simple and systematic, but it can be computationally expensive, especially when dealing with a large number of hyperparameters.

Random Search: Instead of exhaustively searching through all possible combinations, random search randomly samples hyperparameters from specified ranges. This approach can be more efficient than grid search since it doesn't try every possible combination and can lead to better results if performed over a sufficient number of trials.

Bayesian Optimization: Bayesian optimization is a probabilistic model-based approach to hyperparameter tuning. It builds a probabilistic model of the objective function (usually the model's performance metric) and then selects the next set of hyperparameters to evaluate based on this model. It's more efficient than grid search and random search as it adapts over time to focus on promising regions of the hyperparameter space.

Optuna: Optuna is a Python library that provides an automated and efficient approach to hyperparameter tuning using Bayesian optimization. It allows you to define a search space for hyperparameters and then uses algorithms like TPE (Tree-structured Parzen Estimator) to find optimal hyperparameters efficiently.

Hyperopt: Hyperopt is another Python library that uses a combination of random search and Bayesian optimization techniques to tune hyperparameters. It uses a parallel and asynchronous approach to speed up the optimization process.

Genetic Algorithms: Genetic algorithms are inspired by the process of natural selection. They involve creating a population of hyperparameter combinations, evaluating their performance, and then selecting and evolving the best-performing combinations over several generations.

Gradient-Based Optimization: Some hyperparameters can be treated as continuous variables, and gradient-based optimization techniques like gradient descent can be used to optimize them. However, this method is usually more applicable to neural network architectures and specific hyperparameters that have a smooth impact on the model's performance.

Automated Machine Learning (AutoML) Platforms: There are AutoML platforms, such as Google AutoML, H2O.ai, and AutoKeras, that automate the process of hyperparameter tuning along with other tasks like feature engineering and model selection.

# Grid Search

GridSearch is a hyperparameter tuning method that involves defining a grid of possible values for each hyperparameter and then exhaustively searching through all possible combinations of these values to find the best set of hyperparameters.

GridSearchCV(estimator, param_grid, *, scoring=None, n_jobs=None, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)[source]

In [6]:
import pandas as pd
from sklearn import datasets
iris = datasets.load_iris()
df=pd.DataFrame(iris.data,columns=iris.feature_names)
df["flower"]=iris.target
df["flower"]=df["flower"].apply(lambda x:iris.target_names[x])

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: flower, Length: 150, dtype: object

In [15]:
from sklearn.model_selection import train_test_split
Xtrain,Xtest,ytrain,ytest=train_test_split(iris.data,iris.target,train_size=0.3)

In [16]:
from sklearn.svm import SVC
model=SVC(kernel="rbf",C=30,gamma="auto")
model.fit(Xtrain,ytrain)
model.score(Xtest,ytest)

0.9523809523809523

In [17]:
from sklearn.model_selection import cross_val_score

In [18]:
cross_val_score(SVC(kernel="rbf",C=30,gamma="auto"),iris.data,iris.target,cv=5)

array([0.96666667, 1.        , 0.9       , 0.93333333, 1.        ])

In [19]:
cross_val_score(SVC(kernel="linear",C=30,gamma="auto"),iris.data,iris.target,cv=5)

array([1. , 1. , 0.9, 0.9, 1. ])

In [20]:
cross_val_score(SVC(kernel="rbf",C=10,gamma="auto"),iris.data,iris.target,cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [23]:
import numpy as np
kernels=["rbf","linear","poly"]
C=[10,20,30]
avg_score={}
for kval in kernels:
    for cval in C:
        cv_score=cross_val_score(SVC(kernel=kval,C=cval,gamma="auto"),iris.data,iris.target,cv=5)
        avg_score[kval+'_'+str(cval)]=np.average(cv_score)
avg_score

{'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'rbf_30': 0.96,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666,
 'linear_30': 0.96,
 'poly_10': 0.9666666666666666,
 'poly_20': 0.9533333333333334,
 'poly_30': 0.9533333333333334}

In [37]:
from sklearn.model_selection import GridSearchCV
grid=GridSearchCV(SVC(gamma="auto"),{
    "C":[10,20,30],
    "kernel":["rbf","linear","poly"],
},
                  cv=5,
                  return_train_score=False,
                 )
grid.fit(iris.data,iris.target)
df=pd.DataFrame(grid.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000000,0.000000,0.004087,0.005198,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.002079,0.003112,0.000000,0.000000,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.000000,0.900000,0.966667,1.0,0.973333,0.038873,2
2,0.005475,0.006024,0.000000,0.000000,10,poly,"{'C': 10, 'kernel': 'poly'}",1.000000,1.000000,0.900000,0.933333,1.0,0.966667,0.042164,4
3,0.001600,0.003200,0.000000,0.000000,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.000000,0.900000,0.966667,1.0,0.966667,0.036515,3
4,0.002901,0.005802,0.000000,0.000000,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.000000,0.900000,0.933333,1.0,0.966667,0.042164,4
5,0.006328,0.007753,0.003129,0.006258,20,poly,"{'C': 20, 'kernel': 'poly'}",0.966667,0.966667,0.900000,0.933333,1.0,0.953333,0.033993,8
6,0.000000,0.000000,0.000000,0.000000,30,rbf,"{'C': 30, 'kernel': 'rbf'}",0.966667,1.000000,0.900000,0.933333,1.0,0.960000,0.038873,6
7,0.000000,0.000000,0.000000,0.000000,30,linear,"{'C': 30, 'kernel': 'linear'}",1.000000,1.000000,0.900000,0.900000,1.0,0.960000,0.048990,6
8,0.009375,0.012500,0.000000,0.000000,30,poly,"{'C': 30, 'kernel': 'poly'}",0.966667,0.966667,0.900000,0.933333,1.0,0.953333,0.033993,8


In [29]:
df[["param_C","param_kernel","mean_test_score"]]

,param_C,param_kernel,mean_test_score
0,10,rbf,0.980000
1,10,linear,0.973333
2,10,poly,0.966667
3,20,rbf,0.966667
4,20,linear,0.966667
5,20,poly,0.953333
6,30,rbf,0.960000
7,30,linear,0.960000
8,30,poly,0.953333


In [31]:
dir(grid)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_estimator_type',
 '_format_results',
 '_get_default_requests',
 '_get_metadata_request',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run_search',
 '_select_best_index',
 '_validate_data',
 '_validate_params',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 '

In [32]:
grid.best_score_

0.9800000000000001

In [33]:
grid.best_params_

{'C': 10, 'kernel': 'rbf'}

## Random Search:

Random Search is another hyperparameter tuning method that involves randomly sampling hyperparameters from specified ranges, rather than exhaustively searching through all possible combinations like GridSearch. Random Search is more efficient than GridSearch in terms of computational resources, and it can often find good hyperparameter combinations with fewer trials.

class sklearn.model_selection.RandomizedSearchCV(estimator, param_distributions, *, n_iter=10, scoring=None, n_jobs=None, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', random_state=None, error_score=nan, return_train_score=False

In [40]:
from sklearn.model_selection import RandomizedSearchCV
rfc=RandomizedSearchCV(SVC(gamma="auto"),{
    "C":[10,20,30],
    "kernel":["rbf","linear","poly"],
},
                  cv=5,
                  return_train_score=False,
                       n_iter=3
                      )
rfc.fit(iris.data,iris.target)
df=pd.DataFrame(grid.cv_results_)
df[["param_C","param_kernel","mean_test_score"]]

,param_C,param_kernel,mean_test_score
0,10,rbf,0.980000
1,10,linear,0.973333
2,10,poly,0.966667
3,20,rbf,0.966667
4,20,linear,0.966667
5,20,poly,0.953333
6,30,rbf,0.960000
7,30,linear,0.960000
8,30,poly,0.953333


# How to choose best model

# Grid Search vs Random Search - Which One To Use?

Use grid search if you already have a ballpark range of known hyperparameter values that will perform well. Make sure to keep your parameter space small, because grid search can be extremely time-consuming.

Use random search on a broad range of values if you don’t already have an idea of the parameters that will perform well on your model. Random search is faster than grid search and should always be used when you have a large parameter space.

It is also a good idea to use both random search and grid search to get the best possible results.

You can use random search first with a large parameter space since it is faster. Then, use the best hyperparameters found by random search to narrow down the parameter grid, and feed a smaller range of values to grid search.

these lines are taken from:https://www.kdnuggets.com/2022/10/hyperparameter-tuning-grid-search-random-search-python.html